In [1]:
"""Implementations of algorithms for continuous control."""
import functools
from jaxrl_m.typing import *

import jax
import jax.lax as lax
import jax.numpy as jnp
import numpy as np
import optax
from jaxrl_m.common import TrainState, target_update, nonpytree_field
from jaxrl_m.networks import Policy, Critic, ensemblize

import flax
import flax.linen as nn
from functools import partial

NUM_ROLLOUTS = 10

class Temperature(nn.Module):
    initial_temperature: float = 0.01

    @nn.compact
    def __call__(self) -> jnp.ndarray:
        log_temp = self.param('log_temp',
                              init_fn=lambda key: jnp.full(
                                  (), jnp.log(self.initial_temperature)))
        return jnp.exp(log_temp)

class SACAgent(flax.struct.PyTreeNode):
    rng: PRNGKey
    critic: TrainState
    target_critic: TrainState
    actor: TrainState
    temp: TrainState
    config: dict = nonpytree_field()
    
    
    @jax.jit    
    def reset_critic_optimizer(agent):
    
        new_opt_state = agent.critic.tx.init(agent.critic.params)
        new_critic = agent.critic.replace(opt_state=new_opt_state)
        
        return agent.replace(critic=new_critic)

        
        
    @partial(jax.jit,static_argnames=('num_batches',))  
    def update_critic2(agent, transitions: Batch,idxs:jnp.array,num_steps:int):
        
        def one_update(agent, batch: Batch):
                
            new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        
            # def critic_loss_fn(critic_params):
            #     next_dist = agent.actor(batch['next_observations'])
            #     next_actions, next_log_probs = next_dist.sample_and_log_prob(seed=next_key)

            #     next_q1, next_q2 = agent.target_critic(batch['next_observations'], next_actions,True,
            #                                         params=None,rngs={'dropout': next_key})
            #     next_q = jnp.minimum(next_q1, next_q2)
            #     target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q

            #     if agent.config['backup_entropy']:
            #         target_q = target_q - agent.config['discount'] * batch['masks'] * next_log_probs * agent.temp()
                
                
            #     q1, q2 = agent.critic(batch['observations'], batch['actions'],True,
            #                                         params=critic_params,rngs={'dropout': curr_key},
            #                                         )
            #     critic_loss = ((q1 - target_q)**2 + (q2 - target_q)**2).mean()
                
            #     return critic_loss, {
            #         'critic_loss': critic_loss,
            #         'q1': q1.mean(),
            #     }  
            
            def critic_loss_fn(critic_params):
                    
                    
                    next_dist = agent.actor(batch['next_observations'])
                    next_actions, next_log_probs = next_dist.sample_and_log_prob(seed=next_key)

                    concat_actions = jnp.concatenate([batch["actions"],next_actions])
                    concat_observations = jnp.concatenate([batch["observations"],batch["next_observations"]])
                    
                    concat_q = agent.critic(concat_observations, concat_actions,
                                            True,params=critic_params)
                    q,next_q = jnp.split(concat_q,2,axis=0) ## axis=1 for ensemble
                    target_q = batch['rewards'] + agent.config['discount'] * batch['masks'] * next_q
                    
                    # if agent.config['backup_entropy']:
                    #     target_q = target_q - agent.config['discount'] * batch['masks'] * next_log_probs * agent.temp()
                    target_q = jax.lax.stop_gradient(target_q)
                    critic_loss = ((target_q-q)**2).mean()
                    
                    return critic_loss, {
                    'critic_loss': critic_loss,
                    'q1': q.mean(),
                }  
            
            new_critic, critic_info = agent.critic.apply_loss_fn(loss_fn=critic_loss_fn, has_aux=True)
            new_target_critic = target_update(agent.critic, agent.target_critic, agent.config['target_update_rate'])
            return agent.replace(rng=new_rng, critic=new_critic, target_critic=new_target_critic)
        
        
        get_batch = lambda transitions,idx : jax.tree_map(lambda x : x[idx],transitions)
        
        agent = jax.lax.fori_loop(0, num_steps, 
                        lambda i, agent: one_update(agent,get_batch(transitions,idxs[i])),
                        agent)
        
        return agent,{}
    

        
    @jax.jit
    def update_actor(agent, batch: Batch):
        new_rng, curr_key, next_key = jax.random.split(agent.rng, 3)

        def actor_loss_fn(actor_params):
            observations = jnp.repeat(batch['observations'], 10, axis=0)
            discounts = jnp.repeat(batch['discounts'], 10, axis=0)
            masks = jnp.int32(jnp.repeat(batch['masks'], 10, axis=0))

            dist = agent.actor(observations, params=actor_params)
            actions, log_probs = dist.sample_and_log_prob(seed=curr_key)
            q = agent.critic(observations, actions)
            actor_loss = (discounts*(log_probs * agent.temp() - q)).sum()
            
            return actor_loss, {
                'actor_loss': actor_loss,
                'entropy': -1 * ((discounts*log_probs)/jnp.sum(discounts)).sum(),
                #'entropy2': -1*dist.entropy().mean(),
            }
        
        def temp_loss_fn(temp_params, entropy, target_entropy):
            temperature = agent.temp(params=temp_params)
            temp_loss = (temperature * ((entropy - target_entropy)**3)).mean()
            return temp_loss, {
                'temp_loss': temp_loss,
                'temperature': temperature,
            }

        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=actor_loss_fn, has_aux=True)
        temp_loss_fn = functools.partial(temp_loss_fn, entropy=actor_info['entropy'], target_entropy=agent.config['target_entropy'])
        new_temp, temp_info = agent.temp.apply_loss_fn(loss_fn=temp_loss_fn, has_aux=True)
        agent = agent.replace(rng=new_rng, temp=new_temp)
        new_actor, actor_info = agent.actor.apply_loss_fn(loss_fn=actor_loss_fn, has_aux=True)
        #new_temp.params["log_temp"] = jnp.clip(new_temp.params["log_temp"],jnp.log(0.001),jnp.log(1000.0))
        #return agent.replace(rng=new_rng, actor=new_actor, temp=new_temp), {**actor_info, **temp_info}
        return agent.replace(rng=new_rng, actor=new_actor), {**actor_info, **temp_info}

    @jax.jit
    def sample_actions(agent,   
                       observations: np.ndarray,
                       seed: PRNGKey,
                       random = bool,
                       temperature: float = 1.0,
                       ) -> jnp.ndarray:
        actions = agent.actor(observations, temperature=temperature).sample(seed=seed)
        #actions = jnp.clip(actions, -1, 1)
        return actions



def create_learner(
                 seed: int,
                 observations: jnp.ndarray,
                 actions: jnp.ndarray,
                 actor_lr: float = 3e-4,
                 critic_lr: float = 3e-4,
                 temp_lr: float =3e-1,## Test
                 hidden_dims: Sequence[int] = (256, 256),
                 discount: float = 0.99,
                 tau: float = 0.005,
                 target_entropy: float = None,
                 backup_entropy: bool = True,
            **kwargs):

        print('Extra kwargs:', kwargs)

        rng = jax.random.PRNGKey(seed)
        rng, actor_key, critic_key = jax.random.split(rng, 3)

        action_dim = actions.shape[-1]
        actor_def = Policy((256,256), action_dim=action_dim, 
            log_std_min=-10.0, state_dependent_std=True, tanh_squash_distribution=True, final_fc_init_scale=1.0)

        actor_params = actor_def.init(actor_key, observations)['params']
        actor = TrainState.create(actor_def, actor_params, tx=optax.adam(learning_rate=3e-4))
        #actor = TrainState.create(actor_def, actor_params, tx=optax.sgd(learning_rate=1e-3))

        #critic_def = ensemblize(Critic, num_qs=2,split_rngs={"dropout":True})(hidden_dims)
        critic_def = Critic(hidden_dims)
        critic_params = critic_def.init(critic_key, observations, actions)['params']
        critic = TrainState.create(critic_def, critic_params, tx=optax.adam(learning_rate=critic_lr))
        target_critic = TrainState.create(critic_def, critic_params)

        temp_def = Temperature()
        temp_params = temp_def.init(rng)['params']
        temp = TrainState.create(temp_def, temp_params, tx=optax.adam(learning_rate=3e-2))

        if target_entropy is None:
            target_entropy = -0.5 * action_dim

        config = flax.core.FrozenDict(dict(
            discount=discount,
            target_update_rate=tau,
            target_entropy=target_entropy,
            backup_entropy=backup_entropy,            
        ))

        return SACAgent(rng, critic=critic, target_critic=target_critic, actor=actor, temp=temp, config=config)



/home/mahdi/Desktop/supersac/.venv/lib/python3.10/site-packages/jax/_src/api_util.py:229: SyntaxWarning: Jitted function has invalid argnames {'num_batches'} in static_argnames. Function does not take these args.This warning will be replaced by an error after 2022-08-20 at the earliest.
  warnings.warn(f"Jitted function has invalid argnames {invalid_argnames} "


In [2]:

from jaxrl_m.rollout import PolicyRollout


def f(anc_agent,obs,actor,seed):

    dist = anc_agent.actor(obs, params=actor)
    actions, _ = dist.sample_and_log_prob(seed=seed)
    q = anc_agent.critic(obs, actions,params=anc_agent.target_critic.params)
    
    return q
    

@jax.jit
def estimate_return(anc_agent,anc_return,acq_rollout:PolicyRollout,seed):
    
    # acq_obs = acq_rollout.observations
    # acq_masks = acq_rollout.disc_masks
    acq_obs = jnp.repeat(acq_rollout.observations,10,axis=0)
    acq_masks = jnp.repeat(acq_rollout.disc_masks,10,axis=0)
    
    acq_actor = acq_rollout.policy_params
    acq_return = acq_rollout.policy_return
    anc_actor = anc_agent.actor.params
    
    acq_q = f(anc_agent,acq_obs,acq_actor,seed)
    anc_q = f(anc_agent,acq_obs,anc_actor,seed)
    
    adv = ((acq_q - anc_q)*acq_masks).sum()/(acq_rollout.num_rollouts *10)
    acq_return_pred = anc_return + adv
    
    return acq_return_pred,acq_return


def evaluate_critic(anc_agent,anc_return,policy_rollouts,seed):

    y_pred,y= [],[]
    for policy_rollout in policy_rollouts:
        
        acq_return_pred,acq_return = estimate_return(anc_agent,anc_return,policy_rollout,seed)
        y_pred.append(acq_return_pred),y.append(acq_return)
        
    y_pred,y = np.array(y_pred),np.array(y)
    a2 = jnp.clip(((y-y_pred)**2),a_min=1e-4).sum()
    b2=((y-y.mean())**2).sum()
    R2 = 1-(a2/b2)  
    
    return R2


In [3]:
import os
from functools import partial
import numpy as np
import jax
import tqdm
import gymnasium as gym


from jaxrl_m.wandb import setup_wandb, default_wandb_config, get_flag_dict
import wandb
from jaxrl_m.evaluation import supply_rng, evaluate, flatten, EpisodeMonitor
from jaxrl_m.dataset import ReplayBuffer
from collections import deque
from jax import config
# config.update("jax_debug_nans", True)
# config.update("jax_enable_x64", True)
        
from jaxrl_m.rollout import rollout_policy2,rollout_policy

env_name='HalfCheetah-v4'
seed=np.random.choice(1000000)
eval_episodes=10
batch_size = 256
max_steps = int(1e6)
start_steps = 10000                   
log_interval = 6000
#eval_interval = 10000

wandb_config = default_wandb_config()
wandb_config.update({
    'project': 'on_policy_sac',
    'group': 'sac_test',
    'name': 'sac_{env_name}',
})


env = EpisodeMonitor(gym.make(env_name,max_episode_steps=600))
eval_env = EpisodeMonitor(gym.make(env_name))
setup_wandb({"bonjour":1})

example_transition = dict(
    observations=env.observation_space.sample(),
    actions=env.action_space.sample(),
    rewards=0.0,
    masks=1.0,
    next_observations=env.observation_space.sample(),
    discounts=1.0,
)

replay_buffer = ReplayBuffer.create(example_transition, size=int(1e6))
actor_buffer = ReplayBuffer.create(example_transition, size=int(10e3))

agent = create_learner(seed,
                example_transition['observations'][None],
                example_transition['actions'][None],
                max_steps=max_steps,
                #**FLAGS.config
                )

exploration_metrics = dict()
obs,info = env.reset()    
exploration_rng = jax.random.PRNGKey(0)
i = 0
unlogged_steps = 0
policy_rollouts = deque([], maxlen=20)
warmup = True
with tqdm.tqdm(total=max_steps) as pbar:
    
    while (i < max_steps):

        warmup=(i < start_steps)
        replay_buffer,actor_buffer,policy_rollout,policy_return,undisc_policy_return,num_steps = rollout_policy(
                                                                agent,env,exploration_rng,
                                                                replay_buffer,actor_buffer,warmup=warmup,
                                                                num_rollouts=NUM_ROLLOUTS,random=False,
                                                                )
        
        if not warmup : policy_rollouts.append(policy_rollout)
        unlogged_steps += num_steps
        i+=num_steps
        pbar.update(num_steps)
        
            
        if replay_buffer.size > start_steps and len(policy_rollouts)>0:
        
         
            transitions = replay_buffer.get_all()
            idxs = jax.random.choice(a=replay_buffer.size, shape=(10000,512), replace=True,key=jax.random.PRNGKey(0))
            agent.reset_critic_optimizer()
            agent, critic_update_info = agent.update_critic2(transitions,idxs,10000)
            R2 = evaluate_critic(agent,policy_rollouts[-1].policy_return,policy_rollouts,agent.rng)
            
            actor_batch = actor_buffer.get_all()      
            agent, actor_update_info = agent.update_actor(actor_batch)    
            update_info = {**critic_update_info, **actor_update_info, 'R2_validation': R2}
            
            if unlogged_steps > log_interval:
                
               
                policy_fn = partial(supply_rng(agent.sample_actions), temperature=0.0)
                eval_info = evaluate(policy_fn, eval_env, num_episodes=eval_episodes)
                eval_metrics = {f'evaluation/{k}': v for k, v in eval_info.items()}
                exploration_metrics = {f'exploration/disc_return': policy_return}
                train_metrics = {f'training/{k}': v for k, v in update_info.items()}
                train_metrics['training/undisc_return'] = undisc_policy_return
                wandb.log(exploration_metrics, step=int(i),commit=False)
                wandb.log(train_metrics, step=int(i),commit=False)
                wandb.log(eval_metrics, step=int(i),commit=True)
                unlogged_steps = 0


2024-01-24 23:13:40.678504: W external/xla/xla/service/gpu/nvptx_compiler.cc:679] The NVIDIA driver's CUDA version is 12.2 which is older than the ptxas CUDA version (12.3.103). Because the driver is older than the ptxas version, XLA is disabling parallel compilation, which may slow down compilation. You should update your NVIDIA driver or use the NVIDIA-provided CUDA forward compatibility packages.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: mahdikallel. Use `wandb login --relogin` to force relogin


Extra kwargs: {'max_steps': 1000000}


 54%|█████▍    | 540000/1000000 [07:45<07:09, 1071.31it/s]